In [1]:
!pip install transformers
!pip install datasets
!pip install performer-pytorch
!pip install sentencepiece
!pip install wandb
!pip install linformer-pytorch

     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/s 
     |████████████████████████████████| 636 kB 41.5 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 270 kB 5.2 MB/s 
     |████████████████████████████████| 125 kB 51.5 MB/s 
     |████████████████████████████████| 243 kB 56.8 MB/s 
     |████████████████████████████████| 1.3 MB 39.8 MB/s 
     |████████████████████████████████| 294 kB 64.5 MB/s 
     |████████████████████████████████| 142 kB 62.5 MB/s 
  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-py3-none-any.whl size=2900 sha256=67608ab3fa37d0ddffb666bca884d79664d21940a5a7eb0c9e8b0d9eb1e5dcbb
  Stored in directory: /root

In [23]:
from datasets import load_dataset
import torch
import datasets
import transformers
import pandas as pd
import numpy as np
from transformers import BigBirdTokenizer, \
BigBirdForSequenceClassification, Trainer, TrainingArguments,EvalPrediction, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import wandb
import random

In [ ]:
import torch
torch.cuda.empty_cache()

In [24]:
train_data, test_data = load_dataset('glue','sst2', split=['train', 'test'])

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
train_data

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [27]:
for example in train_data:
  print(example['sentence'])
  print("----")

Streaming output truncated to the last 5000 lines.
----
is to substitute plot for personality . 
----
an overstylized , puréed mélange of sex , psychology , drugs and philosophy . 
----
, it does n't disappoint . 
----
its courageousness , and 
----
incapable of being boring 
----
this sad , compulsive life 
----
i saw this one 
----
director/co-writer jacques audiard , though little known in this country , belongs in the very top rank of french filmmakers 
----
paints an absurdly simplistic picture 
----
treasure and 
----
was particularly funny 
----
is in need of a scented bath 
----
the park 
----
like running out screaming 
----
through the pitfalls of incoherence and redundancy 
----
if damon and affleck attempt another project greenlight , next time out they might try paying less attention to the miniseries and more attention to the film it is about . 
----
a must-see 
----
be an unendurable viewing experience for this ultra-provincial new yorker if 26-year-old reese witherspoon

KeyboardInterrupt: ignored

In [ ]:
test_data

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1821
})

# Bigbird

In [ ]:
# https://jesusleal.io/2021/05/14/Big-Bird-text-classification-tutorial/
modelBigBird = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base',
                gradient_checkpointing=False,
                num_labels = 2,
                return_dict=True)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/489M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassifica

In [ ]:
tokenizerBigBird = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', 
                                          max_length = 1024,)

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

In [ ]:
def tokenization(batched_text):
    return tokenizerBigBird(batched_text['sentence'], padding = 'max_length', truncation=True, max_length = 1024)

train_data_big_bird = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data_big_bird = test_data.map(tokenization, batched = True, batch_size = len(test_data))

# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
training_args_big_bird = TrainingArguments(
    output_dir='/content/drive/results',          # output directory
    do_predict= True, 
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=160,                # number of warmup steps for learning rate  
    save_steps=1000,
    save_total_limit=10,
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=4,
    evaluation_strategy = "steps")

# instantiate the trainer class and check for available devices
trainerBigBird = Trainer(
    model=modelBigBird,
    args=training_args_big_bird,
    compute_metrics=compute_metrics,
    train_dataset=train_data_big_bird,
    eval_dataset=test_data_big_bird
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
trainerBigBird.train()

The following columns in the training set  don't have a corresponding argument in `BigBirdForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running training *****
  Num examples = 67349
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 202047


RuntimeError: ignored

# Performer

In [ ]:
from performer_pytorch import PerformerLM

In [ ]:
modelPerformer = PerformerLM(
    num_tokens = 20000,
    max_seq_len = 2048,             # max sequence length
    dim = 512,                      # dimension
    depth = 12,                     # layers
    heads = 8,                      # heads
    causal = False,                 # auto-regressive or not
    nb_features = 256,              # number of random features, if not set, will default to (d * log(d)), where d is the dimension of each head
    feature_redraw_interval = 1000, # how frequently to redraw the projection matrix, the more frequent, the slower the training
    generalized_attention = False,  # defaults to softmax approximation, but can be set to True for generalized attention
    kernel_fn = nn.ReLU(),          # the kernel function to be used, if generalized attention is turned on, defaults to Relu
    reversible = True,              # reversible layers, from Reformer paper
    ff_chunks = 10,                 # chunk feedforward layer, from Reformer paper
    use_scalenorm = False,          # use scale norm, from 'Transformers without Tears' paper
    use_rezero = False,             # use rezero, from 'Rezero is all you need' paper
    #tie_embedding = False,          # multiply final embeddings with token weights for logits, like gpt decoder
    ff_glu = True,                  # use GLU variant for feedforward
    emb_dropout = 0.1,              # embedding dropout
    ff_dropout = 0.1,               # feedforward dropout
    attn_dropout = 0.1,             # post-attn dropout
    local_attn_heads = 4,           # 4 heads are local attention, 4 others are global performers
    local_window_size = 256,        # window size of local attention
    rotary_position_emb = True      # use rotary positional embedding, which endows linear attention with relative positional encoding with no learned parameters. should always be turned on unless if you want to go back to old absolute positional encoding
)

/usr/local/lib/python3.7/dist-packages/performer_pytorch/performer_pytorch.py:131: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:1940.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)


In [ ]:
training_args_performer = TrainingArguments(
    output_dir='/content/drive/results',          # output directory
    do_predict= True, 
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=160,                # number of warmup steps for learning rate  
    save_steps=1000,
    save_total_limit=10,
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=4,
    evaluation_strategy = "steps")

# instantiate the trainer class and check for available devices
trainerPerformer = Trainer(
    model=modelPerformer,
    args=training_args_performer,
    compute_metrics=compute_metrics,
    train_dataset=train_data_big_bird,
    eval_dataset=test_data_big_bird
)


NameError: ignored

In [ ]:
trainerPerformer.train()

# Linformer

In [ ]:
from linformer_pytorch import Linformer
import torch

modelLinformer = Linformer(
        input_size=262144, # Dimension 1 of the input
        channels=64, # Dimension 2 of the input
        dim_d=None, # Overwrites the inner dim of the attention heads. If None, sticks with the recommended channels // nhead, as in the "Attention is all you need" paper
        dim_k=128, # The second dimension of the P_bar matrix from the paper
        dim_ff=128, # Dimension in the feed forward network
        dropout_ff=0.15, # Dropout for feed forward network
        nhead=4, # Number of attention heads
        depth=2, # How many times to run the model
        dropout=0.1, # How much dropout to apply to P_bar after softmax
        activation="gelu", # What activation to use. Currently, only gelu and relu supported, and only on ff network.
        checkpoint_level="C0", # What checkpoint level to use. For more information, see below.
        parameter_sharing="layerwise", # What level of parameter sharing to use. For more information, see below.
        k_reduce_by_layer=0, # Going down `depth`, how much to reduce `dim_k` by, for the `E` and `F` matrices. Will have a minimum value of 1.
        full_attention=False, # Use full attention instead, for O(n^2) time and space complexity. Included here just for comparison
        include_ff=True, # Whether or not to include the Feed Forward layer
        w_o_intermediate_dim=None, # If not None, have 2 w_o matrices, such that instead of `dim*nead,channels`, you have `dim*nhead,w_o_int`, and `w_o_int,channels`
        ).cuda()

In [ ]:
training_args_linformer = TrainingArguments(
    output_dir='/content/drive/results',          # output directory
    do_predict= True, 
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=160,                # number of warmup steps for learning rate  
    save_steps=1000,
    save_total_limit=10,
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=4,
    evaluation_strategy = "steps")

# instantiate the trainer class and check for available devices
trainerLinformer = Trainer(
    model=modelLinerformer,
    args=training_args_linformer,
    compute_metrics=compute_metrics,
    train_dataset=train_data_big_bird,
    eval_dataset=test_data_big_bird
)

In [ ]:
trainerLinformer.train()

# Longformer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizerLongformer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
modelLongformer = AutoModelForSequenceClassification.from_pretrained('allenai/longformer-base-4096', gradient_checkpointing=False,
                num_labels = 2,
                return_dict=True)

In [ ]:
training_args_longformer = TrainingArguments(
    output_dir='/content/drive/results',          # output directory
    do_predict= True, 
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=160,                # number of warmup steps for learning rate  
    save_steps=1000,
    save_total_limit=10,
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=4,
    evaluation_strategy = "steps")

# instantiate the trainer class and check for available devices
trainerLongformer = Trainer(
    model=modelLongformer,
    args=training_args_longformer,
    compute_metrics=compute_metrics,
    train_dataset=train_data_big_bird,
    eval_dataset=test_data_big_bird
)

In [ ]:
trainerLongerformer.train()

# Base Inference (training for huggingface models)

In [14]:
# Big bird
!export GLUE_DIR=/content/SST2
!export TASK_NAME=SST2

!python run_glue.py \
  --model_name_or_path google/bigbird-roberta-base \
  --task_name sst2 \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_gpu_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir /tmp/$TASK_NAME/ \
  --overwrite_output_dir

10/03/2021 16:09:08 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/03/2021 16:09:08 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_mode

In [20]:
!export GLUE_DIR=/content/SST2
!export TASK_NAME=SST2

!python run_glue.py \
  --model_name_or_path 'allenai/longformer-base-4096' \
  --task_name sst2 \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_gpu_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir /tmp/$TASK_NAME/ \
  --overwrite_output_dir

10/03/2021 17:42:05 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/03/2021 17:42:05 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_mode

In [21]:
import datetime
total = 0
correct = 0

start = datetime.timedelta.total_seconds()
m = nn.Softmax(dim=1)
for row in test_data_big_bird:
  total += 1
  output = modelPerformer(row)
  output = m(output)
  if output[0][0] > output[0][1]:
    if row['label'] == 0:
      correct += 1
  else:
    if row['label'] == 1:
      correct += 1
end = datetime.timedelta.total_seconds()
time_delta = (end - start)
total_seconds = time_delta.total_seconds()
minutes = total_seconds/60

print(minutes)
print(1.0 * correct / total)


SyntaxError: ignored